In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import category_encoders as ce
import pickle
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.neural_network import *
from sklearn.multioutput import *
from sklearn.ensemble import *

In [2]:
with open("../data/processed/train.pkl", "rb") as f:
    train = pickle.load(f)

In [3]:
with open("../data/processed/target.pkl", "rb") as f:
    target = pickle.load(f)

In [4]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког', 'Частота пасс кур']

In [36]:
train.dtypes

Пол                                       object
Семья                                     object
Этнос                                     object
Национальность                            object
Религия                                   object
Образование                               object
Профессия                                 object
Вы работаете?                               int8
Выход на пенсию                             int8
Прекращение работы по болезни               int8
Сахарный диабет                             int8
Гепатит                                     int8
Онкология                                   int8
Хроническое заболевание легких              int8
Бронжиальная астма                          int8
Туберкулез легких                           int8
ВИЧ/СПИД                                    int8
Регулярный прим лекарственных средств       int8
Травмы за год                               int8
Переломы                                    int8
Статус Курения      

In [5]:
train[CAT_COLS] = train[CAT_COLS].astype('object')

In [6]:
train_data, val_data, train_target, val_target = train_test_split(train, target, train_size=0.8, random_state=42)

In [7]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

In [8]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [10]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, REAL_COLS),
    ('cat_cols', cat_pipe, CAT_COLS),
    ('woe_cat_cols', ce.WOEEncoder(), CAT_COLS),
    ('ohe_cols', 'passthrough', OHE_COLS)
]
)

In [11]:
model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', RandomForestClassifier())
]
)

In [12]:
multiout_model_pipe = MultiOutputClassifier(model_pipe, n_jobs=4)

In [13]:
fbeta_score_recall = make_scorer(fbeta_score, beta=2.0, average='micro')

In [14]:
params = {'estimator__model__n_estimators': np.arange(10, 100, 5)}

In [15]:
scores = RandomizedSearchCV(multiout_model_pipe, params, scoring=fbeta_score_recall, error_score='raise')

In [38]:
type(scores.best_estimator_)

sklearn.multioutput.MultiOutputClassifier

In [16]:
scores.fit(X=train_data, y=train_target)

RandomizedSearchCV(error_score='raise',
                   estimator=MultiOutputClassifier(estimator=Pipeline(steps=[('preprocess',
                                                                              ColumnTransformer(transformers=[('real_cols',
                                                                                                               Pipeline(steps=[('imputer',
                                                                                                                                SimpleImputer()),
                                                                                                                               ('scaler',
                                                                                                                                StandardScaler())]),
                                                                                                               ['Возраст '
                                                                                                                'курения',
                                                                                                                'Сигарет '
                                                                                                                'в '
                                                                                                                'день',
                                                                                                                'Возраст '
                                                                                                                'алког',
                                                                                                                'Частота '
                                                                                                                'пасс '
                                                                                                                'кур']),
                                                                                                              ('cat_cols',
                                                                                                               Pipeline(steps=[('imputer',
                                                                                                                                Si...
                                                                                                                'лекарственных '
                                                                                                                'средств',
                                                                                                                'Травмы '
                                                                                                                'за '
                                                                                                                'год',
                                                                                                                'Переломы',
                                                                                                                'Пассивное '
                                                                                                                'курение',
                                                                                                                'Сон '
                                                                                                                'после '
                                                                                                                'обеда',
                                                                                                                'Спорт, '
                                                                                                

In [17]:
scores.best_score_

0.39783623406755037

In [28]:
cat_boost_model = CatBoostClassifier(loss_function='MultiLogloss',
                silent=True,
                random_seed=42,
                cat_features=CAT_COLS)

In [29]:
grid = {'learning_rate': [0.01, 0.1],
        'depth': [4, 10],
        'l2_leaf_reg': [1, 2, 3, 4, 5],
        'early_stopping_rounds': [50, 100, 150]}

In [30]:
cat_boost_model.randomized_search(grid, train_data, train_target, 4)

/home/kasyanikita/HSE/Lab1_Data_Analysis/venv/lib/python3.8/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3521529231
bestIteration = 459

0:	loss: 0.3521529	best: 0.3521529 (0)	total: 5.53s	remaining: 49.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3491257764
bestIteration = 55

1:	loss: 0.3491258	best: 0.3491258 (1)	total: 7.56s	remaining: 30.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3461290082
bestIteration = 90

2:	loss: 0.3461290	best: 0.3461290 (2)	total: 10.1s	remaining: 23.5s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 0.3504863674
bestIteration = 71

3:	loss: 0.3504864	best: 0.3461290 (2)	total: 13s	remaining: 19.5s
Stopped by overfitting detector  (150 iterations wait)

bestTest = 0.349068411
bestIteration = 138

4:	loss: 0.3490684	best: 0.3461290 (2)	total: 16.5s	remaining: 16.5s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3498248941
bestIteration = 36

5:	loss: 0.3498249	best: 0.3461290 (2)	total: 35.5s	remai

{'params': {'depth': 4,
  'od_wait': 100,
  'l2_leaf_reg': 4,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
  

In [31]:
cat_boost_model.predict(train_data)

array([[0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [34]:
fbeta_score(val_target, cat_boost_model.predict(val_data), beta=2.0, average='micro')

/home/kasyanikita/HSE/Lab1_Data_Analysis/venv/lib/python3.8/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0.43833580980683506